# Multi-Agent System with OpenAI and Mocked External Tools

This notebook demonstrates a **Multi-Agent** approach:
1. **Reflexion Agent** – uses **OpenAI** to generate code for CSV analysis, but *executes code locally* in a mock environment.
2. **Tool-Using Agent** – queries a **mock** internal knowledge base, no real external calls.
3. **Auto-GPT Agent** – calls **OpenAI** for multi-step reasoning about external research, but the "web search" is **mocked**.
4. **Master Orchestrator** – coordinates the sub-agents, merging their outputs into a final answer.

**Goal**: “Perform advanced analysis of EV data from a CSV, gather external info, consult internal knowledge base about EV manufacturing, then produce a final recommendation.”

We rely on the **OpenAI** library for any LLM logic and mock all other external calls. This is **conceptual**—in real usage, you might replace these mocks with actual DB queries, web scraping, or file I/O.

## 1. OpenAI Setup
We'll define a **`call_openai`** function to handle all LLM calls. Make sure you have:
```bash
pip install openai
```
and an environment variable `OPENAI_API_KEY` (or set it explicitly in code).

In [ ]:
from openai import OpenAI
import os
import re
import pandas as pd
from datetime import datetime


os.environ['OPENAI_API_KEY']=''



# Simple adapter to call OpenAI once.
# Adjust 'model' to 'gpt-4' or 'gpt-3.5-turbo' as you prefer.

def call_openai(system_msg, user_msg, model="gpt-4o", temperature=0.2, max_tokens=200):
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]

    client = OpenAI()
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

## 2. Reflexion Agent
We mock:
- **Local CSV data** as a `pandas.DataFrame`.
- Code generation uses **OpenAI** for snippet creation.
- We then **execute** that code in a safe environment.
- If there's an **error**, we reflect with the error message back to OpenAI for a corrected snippet.

### Implementation Steps
1. The agent calls `call_openai` to get an **initial code snippet**.
2. **Executes** the code with Python’s `exec(...)` in a restricted local environment.
3. If an error, it **reflects** by passing the error back to OpenAI for a correction.
4. Repeats until success or we exceed `max_reflections`.

In [ ]:
class ReflexionCSVAgent:
    def __init__(self, max_reflections=2, model="gpt-4o"):
        self.max_reflections = max_reflections
        self.model = model
        self.conversation_log = []

    def analyze_data(self, user_query, df):
        """
        Returns analysis result (dictionary or text).
        """
        # 1) Prompt OpenAI for code snippet
        system_msg = "You are a CSV data analysis assistant using Reflexion."
        user_msg = (
            f"User Query: {user_query}\n"
            "Generate Python code to analyze a pandas DataFrame named 'df'.\n"
            "Store final output in a variable 'result'."
        )
        code_snippet = call_openai(system_msg, user_msg, model=self.model)
        self.conversation_log.append(f"Initial code:\n{code_snippet}\n")

        success, outcome = self.run_code(code_snippet, df)

        reflections = 0
        while not success and reflections < self.max_reflections:
            reflections += 1
            error_msg = outcome
            self.conversation_log.append(f"Error encountered: {error_msg}\n")

            # 2) Reflection step
            reflect_msg = (
                "The code failed with error:\n" + error_msg + "\n"
                "Please correct the code so it works properly."
            )
            corrected_code = call_openai(system_msg, reflect_msg, model=self.model)
            self.conversation_log.append(f"Corrected code:\n{corrected_code}\n")

            success, outcome = self.run_code(corrected_code, df)

        if success:
            return outcome if outcome else "No meaningful result."
        else:
            return "Reflexion agent failed after multiple attempts."    

    def run_code(self, code_str, df):
        # Minimal environment
        local_env = {"df": df, "result": None}
        try:
            exec(code_str, {}, local_env)
            return True, local_env.get("result", "No result variable.")
        except Exception as e:
            return False, str(e)

## 3. Tool-Using Agent
Mocks an **internal knowledge base** about EV manufacturing, battery recycling, etc. No actual external calls. The agent simply returns the relevant text from a dictionary.

In [ ]:
MOCK_KNOWLEDGE_BASE = {
    "EV manufacturing": "Production capacity for new EV models is ramping up. Batteries come from multiple suppliers.",
    "Battery recycling": "Company policy emphasizes eco-friendly recycling programs."
}

class ToolUsingAgent:
    def __init__(self):
        self.conversation_log = []

    def query_internal_kb(self, topic):
        """
        Mocks a knowledge base lookup.
        """
        self.conversation_log.append(f"ToolUsingAgent: Query topic='{topic}'")
        response = MOCK_KNOWLEDGE_BASE.get(topic, "No info available.")
        self.conversation_log.append(f"ToolUsingAgent: answer='{response}'")
        return response

## 4. Auto-GPT Agent
We **mock** the web search and scraping, but for the **agent’s planning** logic, we use **OpenAI** to produce a short plan. In reality, you might do multiple loops. For brevity, we do a **single** LLM call that either references “best EV review” or “price site.”

In [ ]:
MOCK_WEB_DATA = {
    "best_ev_reviews.com": "The Tesla Model Y is widely considered the best EV by consumer sentiment.",
    "ev_price_tracker.com": "Current average price for Tesla Model Y is about $53,000."
}

def mock_web_search(topic):
    sites = []
    tl = topic.lower()
    if "best ev" in tl or "top ev" in tl:
        sites.append("best_ev_reviews.com")
    if "price" in tl:
        sites.append("ev_price_tracker.com")
    if not sites:
        sites.append("general_ev_site.com")
    return sites

def mock_web_scrape(url):
    return MOCK_WEB_DATA.get(url, "No relevant info.")

class AutoGPTAgent:
    def __init__(self, model="gpt-4o"):
        self.model = model
        self.conversation_log = []

    def research_topic(self, topic):
        """
        Uses OpenAI to decide which sites to "visit" (mock), then returns aggregated data.
        """
        system_msg = "You are an Auto-GPT style agent focusing on EV research."        
        user_msg = f"User wants external info on: {topic}. Decide which sites to visit (like best_ev_reviews.com or ev_price_tracker.com)."

        # Let the LLM produce a plan
        plan_response = call_openai(system_msg, user_msg, model=self.model, max_tokens=150)
        self.conversation_log.append(f"Plan: {plan_response}")

        # We’ll do a simple keyword check to see if it suggests a best EV or a price check.
        sites_to_visit = []
        if "best_ev_reviews.com" in plan_response.lower():
            sites_to_visit.append("best_ev_reviews.com")
        if "ev_price_tracker.com" in plan_response.lower():
            sites_to_visit.append("ev_price_tracker.com")

        # fallback
        if not sites_to_visit:
            sites_to_visit = mock_web_search(topic)

        combined_data = []
        for site in sites_to_visit:
            data = mock_web_scrape(site)
            combined_data.append(data)

        return "\n".join(combined_data)

## 5. Multi-Agent Orchestrator
Ties everything together:
1. Calls **ReflexionCSVAgent** to process local CSV.
2. Calls **ToolUsingAgent** to get internal info.
3. Calls **AutoGPTAgent** to do a (mock) external search.
4. Merges results into a final recommendation.


In [ ]:
class MultiAgentOrchestrator:
    def __init__(self, model="gpt-4o"):
        self.reflexion_agent = ReflexionCSVAgent(model=model)
        self.tool_agent = ToolUsingAgent()
        self.autogpt_agent = AutoGPTAgent(model=model)

    def handle_request(self, user_query, df):
        """
        Steps:
        1) Reflexion agent -> analyze CSV.
        2) Tool-Using -> query internal KB.
        3) Auto-GPT -> external research.
        4) Merge final.
        """
        analysis_result = self.reflexion_agent.analyze_data(user_query, df)
        internal_info = self.tool_agent.query_internal_kb("EV manufacturing")
        external_data = self.autogpt_agent.research_topic("Best EV and price")

        return self._combine(analysis_result, internal_info, external_data)

    def _combine(self, analysis_result, internal_info, external_info):
        # Summarize
        if isinstance(analysis_result, dict):
            summary_text = (
                f"Local CSV Analysis:\n"
                f" - Max Range: {analysis_result.get('max_range', 'N/A')}\n"
                f" - Average Price: {analysis_result.get('avg_price', 'N/A')}\n\n"
            )
        else:
            summary_text = f"Local CSV Analysis: {analysis_result}\n\n"

        summary_text += f"Internal Knowledge: {internal_info}\n\n"
        summary_text += f"External Research:\n{external_info}\n\n"

        recommendation = (
            "Based on local CSV data, internal manufacturing knowledge, and external market info,\n"
            "the best EV to focus on is likely the Tesla Model Y, given highest range and strong user sentiment."
        )

        return summary_text + recommendation

## 6. Main
We create a sample CSV DataFrame, set a user query, and run the orchestrator.

In [ ]:
if __name__ == "__main__":
    # Mock CSV data
    sample_data = {
        "Model": ["EV A", "EV B", "Tesla Model Y", "EV C"],
        "Range": [250, 220, 300, 280],
        "Price": [40000, 35000, 50000, 45000]
    }
    df = pd.DataFrame(sample_data)

    # High-level user request
    user_query = "Perform advanced analysis of EV data, get external info, and consult internal knowledge about EV manufacturing." 

    orchestrator = MultiAgentOrchestrator(model="gpt-4o")
    final_answer = orchestrator.handle_request(user_query, df)

    print("=== FINAL ANSWER ===")
    print(final_answer)

## How It Works
1. **ReflexionCSVAgent** uses OpenAI to generate code, runs it locally. If an error occurs, it passes the error message back to OpenAI for a corrected snippet.
2. **ToolUsingAgent** just returns a string from a dictionary—mocking an "internal KB." No external calls.
3. **AutoGPTAgent** calls OpenAI for a quick "plan" on which sites to visit, but the web searching and scraping are **mocked** (`mock_web_search`, `mock_web_scrape`).
4. The **Orchestrator** calls each sub-agent, merges results into a single user-facing message.

## Usage
1. **Install** `openai` (`pip install openai`).
2. **Set** your `OPENAI_API_KEY`.
3. **Run** all cells. The final cell prints the combined answer.

## Key Takeaways
- Only **OpenAI** calls are real (for code gen, reflection, and mini "auto-gpt" steps).
- All “tools” (CSV exec, knowledge base, web scraping) are mocked.
- This design shows how multiple specialized agents can be orchestrated in a single system.